In [12]:
from chainer import Chain, Variable, cuda, functions, links, optimizer, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F 
import chainer.links as L
import numpy as np
import pickle
import MeCab
import traceback
from LSTM_s2s import LSTM_Encoder, LSTM_Decoder, Seq2Seq

EMBED_SIZE = 300
HIDDEN_SIZE = 150
TRAIN_BATCH_SIZE = 40
TEST_BATCH_SIZE = 1

dictpath = "/Users/daisuke/WSL/LSTM/word_id_dict.pickle"
inputpath = "/Users/daisuke/WSL/LSTM/output/S2Smodel_EMBED%s_HIDDEN%s_BATCH%s_EPOCH%s.npz"
inputpath2 = "/Users/daisuke/WSL/LSTM/output_StoU/S2Smodel_EMBED%s_HIDDEN%s_BATCH%s_EPOCH%s.npz"
inputpath3 = "/Users/daisuke/WSL/LSTM/output_annotate/S2Smodel_EMBED%s_HIDDEN%s_BATCH%s_EPOCH%s.npz"
inputpath4 = "/Users/daisuke/WSL/LSTM/output_mix/S2Smodel_EMBED%s_HIDDEN%s_BATCH%s_EPOCH%s.npz"

In [13]:
def test(inputw, epoch, path):
    
    dictf = open(dictpath, 'rb')
    w_id_dict = pickle.load(dictf)
    id_w_dict = {v:k for k, v in w_id_dict.items()}
    
    vocab_size = len(w_id_dict)
    
    model = Seq2Seq(vocab_size = vocab_size,
                    embed_size=EMBED_SIZE,
                    hidden_size=HIDDEN_SIZE,
                    batch_size=TEST_BATCH_SIZE)
    
    inputfile = path%(EMBED_SIZE, HIDDEN_SIZE, TRAIN_BATCH_SIZE, epoch)
    serializers.load_npz(inputfile, model)
    
    mt = MeCab.Tagger("-Ochasen")
    mt.parse('')
    node = mt.parseToNode(inputw)
    inputid_list = []
    while node:
        if node.surface == '':
            node = node.next
            continue
        inputid_list.insert(0,w_id_dict[node.surface])
        node = node.next
        
    enc_words = [Variable(np.array([row], dtype='int32')) for row in inputid_list]
    model.encode(enc_words)
    t = Variable(np.array([0], dtype='int32'))
    
    count = 0
    talk = ""
    while count < 20:
        y = model.decode(t)
        y_list = list(y[0].data)
        y_max = y_list.index(max(y_list))
        
        if id_w_dict[y_max] == 'EOF':
            break
        
        t = Variable(np.array([y_max], dtype='int32'))
        #print(id_w_dict[y_max], end=' ')
        talk += id_w_dict[y_max]
        
        count += 1
    #print()
    return talk


### 人の発話とシステムの応答を対話としたモデルのテスト  

In [17]:
if __name__ == "__main__":
    try:
        utterance = "こんにちは"
        print(utterance)
        for i in range(10):
            print(i+1,end=" ")
            print(test(utterance, i+1, inputpath))
    
    except Exception:
        traceback.print_exc()

こんにちは
1 うん
2 こん
3 こんにちはこんは
4 こんあり
5 こんにちはこんにちはこんにちは
6 こんにちはこんにちはこんにちは
7 こんにちはこんにちはこんにちは
8 こんにちはこんにちはこんにちは
9 こんにちはこんにちはこんにちは
10 こんにちはこんにちは


### システムの発話と人の応答を対話としたモデルのテスト  

In [18]:
if __name__ == "__main__":
    try:
        utterance = "こんにちは"
        print(utterance)
        for i in range(10):
            print(i+1,end=" ")
            print(test(utterance, i+1, inputpath2))
    
    except Exception:
        traceback.print_exc()

こんにちは
1 そうですね。
2 何かな
3 
4 。
5 。
6 なんだ。
7 なんだ。
8 なんだよね
9 なんだぜ
10 なんだ。


### 人の発話とシステムの応答を対話としたモデルのテスト（アノテーション処理）    

In [ ]:
if __name__ == "__main__":
    try:
        utterance = "こんにちは"
        print(utterance)
        for i in range(10):
            print(i+1,end=" ")
            print(test(utterance, i+1, inputpath3))
    
    except Exception:
        traceback.print_exc()

こんにちは
1 そうです。
2 うん
3 うん
4 うん
5 こんにちはうん
6 こんにちはね
7 こんにちはね

In [ ]:
if __name__ == "__main__":
    utterance = "サッカーはお好きですか。"
    i_s = 10
    i_u = 10
    count = 0
    conv_max = 30
    
    conv_s_log = []
    conv_u_log = []
    
    while True:
        print("S'{}:".format(i_s),end=" ")
        utterance = test(utterance, i_s, inputpath3)
        if utterance in conv_s_log:
            i_s = i_s+1 if i_s < 10 else 1
        conv_s_log.append(utterance)
        
        print(utterance)
        print("U'{}:".format(i_u),end=" ")
        utterance = test(utterance, i_u, inputpath3)
        if utterance in conv_u_log:
            i_u = i_u+1 if i_u < 10 else 1
        conv_u_log.append(utterance)
        
        print(utterance)
        
        count += 1
        if count > 30:
            break
        

S'10: サッカーは大好きですね
U'10: サッカーはどうですか。
S'10: サッカーは楽しいですね
U'10: サッカーは楽しいですね
S'10: サッカーは楽しいですね
U'10: サッカーは楽しいですね
S'1: そうですね。
U'1: そうです。
S'1: そうです。
U'1: そうです。
S'1: そうです。
U'2: そうです。
S'2: そうです。
U'3: そうですよ。
S'3: うん
U'3: そうです。
S'3: そうですよ。
U'4: そうですよね。
S'3: うん
U'4: そうです。
S'4: そうですよ。
U'5: そうですよ
S'5: そうですよ
U'5: そうですよ
S'5: そうですよ
U'6: そうですよ
S'6: そうですよ